# 导包

In [1]:
# -*- coding: utf-8 -*-
import json
import re
import math
import os
import threading
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from keras.utils import plot_model
from struct import unpack, pack
import numpy as np
from math import sqrt
import scipy.io as scio
from sklearn.decomposition import PCA
from scipy import interpolate
import scipy.signal as signal
import matplotlib.pyplot as plt
from sklearn import manifold
from tqdm import tqdm
import scipy.io.wavfile as wav
from scipy.fftpack import fft
from random import shuffle
from collections import Counter  
from sklearn.ensemble import IsolationForest 
from sklearn.model_selection import train_test_split
!pip install pyts==0.7.1
from pyts.image import *

!cat /proc/cpuinfo | grep 'model name' |uniq
!nvidia-smi
    
# 设置GPU内存按需分配
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

model name	: Intel(R) Core(TM) i9-10900F CPU @ 2.80GHz
Tue Dec 22 10:27:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3070    Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   36C    P8    10W / 270W |    281MiB /  7979MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                 

# 特征工程

## 提取CSI中想要的部分

# 搭建模型

In [18]:
# 添加CTC损失函数
def ctc_lambda(args):
    labels, y_pred, input_length, label_length = args
    y_pred = y_pred[:, :, :]
    return tf.keras.backend.ctc_batch_cost(labels, y_pred, input_length, label_length)
# 定义解码器


def decode_ctc(preds, py_list):
    window_num = np.zeros((1), dtype=np.int32)
    window_num[0] = preds.shape[1]
    decode = keras.backend.ctc_decode(
        preds, window_num, greedy=True, beam_width=100, top_paths=1)
    result_index = keras.backend.get_value(decode[0][0])[0]
    result_py = []
    for i in result_index:
        try:
            result_py.append(py_list[i])
        except IndexError:
            print(i, 'not in py_list')
    return result_index, result_py


def create_model(input_size, output_size):
    inputs = Input(name='the_inputs', shape=input_size)
    # 1
    h1_1 = Conv2D(64, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_1-1')(inputs)
    h1_2 = BatchNormalization(name='BatchNormal_1-1')(h1_1)
    h1_3 = Conv2D(64, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_1-2')(h1_2)
    h1_4 = BatchNormalization(name='BatchNormal_1-2')(h1_3)
    h1_5 = MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", name='MaxPooling2D_1')(h1_4)
    # 2
    h2_1 = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_2-1')(h1_5)
    h2_2 = BatchNormalization(name='BatchNormal_2-1')(h2_1)
    h2_3 = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_2-2')(h2_2)
    h2_4 = BatchNormalization(name='BatchNormal_2-2')(h2_3)
    h2_5 = MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", name='MaxPooling2D_2')(h2_4)
    # 3
    h3_1 = Conv2D(256, (3, 3), use_bias=True, activation='relu', padding='same',
                  kernel_initializer='he_normal', name='Conv2D_3-1')(h2_5)
    h3_2 = BatchNormalization(name='BatchNormal_3-1')(h3_1)
    h3_3 = Conv2D(256, (3, 3), use_bias=True, activation='relu', padding='same',
                  kernel_initializer='he_normal', name='Conv2D_3-2')(h3_2)
    h3_4 = BatchNormalization(name='BatchNormal_3-2')(h3_3)
    h3_5 = MaxPooling2D(pool_size=(2, 2), strides=None,
                        padding="valid", name='MaxPooling2D_3')(h3_4)
    # 4
    h4_1 = Conv2D(512, (3, 3), use_bias=True, activation='relu', padding='same',
                  kernel_initializer='he_normal', name='Conv2D_4-1')(h3_5)
    h4_2 = BatchNormalization(name='BatchNormal_4-1')(h4_1)
    h4_3 = Conv2D(512, (3, 3), use_bias=True, activation='relu', padding='same',
                  kernel_initializer='he_normal', name='Conv2D_4-2')(h4_2)
    h4_4 = BatchNormalization(name='BatchNormal_4-2')(h4_3)
    # 由于声学模型网络结构原因（3个maxpooling层），我们的音频数据的每个维度需要能够被8整除。这里输入序列经过卷积网络后，长度缩短了8倍，因此我们训练实际输入的数据为wav_len//8。
    h5_1 = Reshape((-1, int(input_size[1]//8*512)), name='Reshape_1')(h4_4)
    lstm_1 = LSTM(128, return_sequences=True, kernel_initializer='he_normal', name='lstm1')(h5_1)
    lstm_2 = LSTM(256, return_sequences=True, kernel_initializer='he_normal', name='lstm2')(lstm_1)
    h5_2 = Dense(512, activation='relu', use_bias=True, kernel_initializer='he_normal', name='Dense_1')(lstm_2)
    h5_3 = Dense(output_size, activation="relu", use_bias=True, kernel_initializer='he_normal', name='Dense_2')(h5_2)  # (layer_h15)

    outputs = Activation('softmax', name='Activation_1')(h5_3)
    base_model = keras.Model(inputs=inputs, outputs=outputs)

    labels = Input(name='the_labels', shape=[None], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    # keras.Lambda(function, output_shape=None, mask=None, arguments=None)
    # 将任意表达式封装为 Layer 对象
    loss_out = Lambda(ctc_lambda, output_shape=(1,), name='ctc')([labels, outputs, input_length, label_length])
    ctc_model = keras.Model(inputs=[labels, inputs, input_length, label_length], outputs=loss_out)

    opt = keras.optimizers.Adam(lr=0.0008, beta_1=0.9, beta_2=0.999, decay=0.01, epsilon=10e-8)
    # ctc_model=multi_gpu_model(ctc_model,gpus=2)
    ctc_model.compile(loss={'ctc': lambda y_true, output: output}, optimizer=opt, metrics=['accuracy'])

    return base_model, ctc_model

# 开始训练、测试

In [ ]:
image_size=64  
batch_size = 16
logs_path = '../logs'

input_size=(None, image_size, 1)
base_model, ctc_model = create_model(input_size, output_size=py_list_size) 
print(ctc_model.summary())

# keras.utils.plot_model(base_model, show_shapes=True)

if not os.path.exists(logs_path):  # 判断保存模型的目录是否存在
    os.makedirs(logs_path)  # 如果不存在，就新建一个，避免之后保存模型的时候炸掉

# train_batch_gen = get_batch_generator(batch_size, train_wav_file_list, train_label_list, py_list, image_size)
# validate_batch_gen = get_batch_generator(batch_size, validate_wav_file_list, validate_label_list, py_list, image_size)
# input_data = next(train_batch_gen)[0]
# plt.imshow(input_data['the_inputs'][0].T[0])
# plt.show()
# print(input_data['the_inputs'].shape, input_data['the_labels'].shape, input_data['input_length'].shape, input_data['label_length'].shape)

train_data = get_train_data(train_wav_file_list, train_label_list, py_list, image_size)

cb = []
cb.append(keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0))
# 当监测值不再改善时，该回调函数将中止训练可防止过拟合
cb.append(keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=50, verbose=1, mode='auto'))
# his = ctc_model.fit_generator(train_batch_gen, verbose=1, steps_per_epoch=train_file_nums//batch_size, validation_data=validate_batch_gen, validation_steps=validate_file_nums//batch_size, epochs=1000, callbacks=cb)  

his = ctc_model.fit(train_data[0], train_data[1], validation_split=0.1, batch_size=128, epochs=1000, callbacks=cb)  # callback的epoch都是对fit里的参数来说

#  保存模型结构及权重
ctc_model.save_weights(r'save_weights.h5')
with open(r'model_struct.json', 'w') as f:
    json_string = base_model.to_json()
    f.write(json_string)  # 保存模型信息
print('模型结构及权重已保存')

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_inputs (InputLayer)         [(None, None, 64, 1) 0                                            
__________________________________________________________________________________________________
Conv2D_1-1 (Conv2D)             (None, None, 64, 64) 640         the_inputs[0][0]                 
__________________________________________________________________________________________________
BatchNormal_1-1 (BatchNormaliza (None, None, 64, 64) 256         Conv2D_1-1[0][0]                 
__________________________________________________________________________________________________
Conv2D_1-2 (Conv2D)             (None, None, 64, 64) 36928       BatchNormal_1-1[0][0]            
___________________________________________________________________________________________

In [ ]:
# 加载权重
with open(r'model_struct.json') as f:
    model_struct = f.read()
test_model = keras.models.model_from_json(model_struct)
test_model.load_weights(r'save_weights.h5')
# model = keras.models.load_model('all_model.h5')
print('模型已加载')
py_list = []
with open(r'pinyin_list.txt', 'r') as f:
    contents = f.readlines()
for line in contents:
    i = line.strip('\n')
    py_list.append(i)
print('py_list已加载')

# 对模型进行评价


def evaluate(kind, wavs, labels):
    data_num = len(wavs)
    error_cnt = 0
    for i in range(data_num):
        pre = test_model.predict(wavs[i])  # (1, 20, 11)
        pre_index, pre_label = decode_ctc(pre, py_list)  # ['5']
        try:
            pre_label = int(pre_label[0])
        except:
            pre_label = None
        label = int(py_list[labels[i]])
        if label != pre_label:
            error_cnt += 1
            print('真实标签：', label, '预测结果', pre_label)
    print('{}:样本数{}错误数{}准确率：{:%}'.format(
        kind, data_num, error_cnt, (1-error_cnt/data_num)))


# 训练集
train_wavs, train_labels = get_test_data(image_size, train_wav_file_list, train_label_list, py_list)
# 测试集
test_wavs, test_labels = get_test_data(image_size, test_wav_file_list, test_label_list, py_list)

evaluate('trian', train_wavs, train_labels)
evaluate('test', test_wavs, test_labels)

#  对比实验

## CNN

In [ ]:
def create_cnn_model(input_size, output_size):
    inputs = Input(name='the_inputs', shape=input_size)
    # 1
    h1_1 = Conv2D(64, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_1-1')(inputs)
    h1_2 = BatchNormalization(name='BatchNormal_1-1')(h1_1)
    h1_3 = Conv2D(64, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_1-2')(h1_2)
    h1_4 = BatchNormalization(name='BatchNormal_1-2')(h1_3)
    h1_5 = MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", name='MaxPooling2D_1')(h1_4)
    # 2
    h2_1 = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_2-1')(h1_5)
    h2_2 = BatchNormalization(name='BatchNormal_2-1')(h2_1)
    h2_3 = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_2-2')(h2_2)
    h2_4 = BatchNormalization(name='BatchNormal_2-2')(h2_3)
    h2_5 = MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", name='MaxPooling2D_2')(h2_4)
    # 3
    h3_1 = Conv2D(256, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_3-1')(h2_5)
    h3_2 = BatchNormalization(name='BatchNormal_3-1')(h3_1)
    h3_3 = Conv2D(256, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_3-2')(h3_2)
    h3_4 = BatchNormalization(name='BatchNormal_3-2')(h3_3)
    h3_5 = MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", name='MaxPooling2D_3')(h3_4)
    # 4
    h4_1 = Conv2D(521, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_4-1')(h3_5)
    h4_2 = BatchNormalization(name='BatchNormal_4-1')(h4_1)
    h4_3 = Conv2D(512, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_4-2')(h4_2)
    h4_4 = BatchNormalization(name='BatchNormal_4-2')(h4_3)
    # 由于声学模型网络结构原因（3个maxpooling层），我们的音频数据的每个维度需要能够被8整除。这里输入序列经过卷积网络后，长度缩短了8倍，因此我们训练实际输入的数据为wav_len//8。
    h5_1 = Reshape((-1, int(input_size[1]//8*512)), name='Reshape_1')(h4_4)
    h5_2 = Dense(512, activation='relu', use_bias=True, kernel_initializer='he_normal', name='Dense_1')(h5_1)
    h5_3 = Dense(output_size, activation="relu", use_bias=True, kernel_initializer='he_normal', name='Dense_2')(h5_2)#(layer_h15)

    outputs = Activation('softmax', name='Activation_1')(h5_3)

    base_model = keras.Model(inputs=inputs, outputs=outputs)

    labels = Input(name='the_labels', shape=[None], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    # keras.Lambda(function, output_shape=None, mask=None, arguments=None)
    # 将任意表达式封装为 Layer 对象
    loss_out = Lambda(ctc_lambda, output_shape=(1,), name='ctc')([labels, outputs, input_length, label_length])
    ctc_model = keras.Model(inputs=[labels, inputs, input_length, label_length], outputs=loss_out)

    opt = keras.optimizers.Adam(lr=0.0008, beta_1=0.9, beta_2=0.999, decay=0.01, epsilon=10e-8)
    # ctc_model=multi_gpu_model(ctc_model,gpus=2)
    ctc_model.compile(loss={'ctc': lambda y_true, output: output}, optimizer=opt, metrics=['accuracy'])

    return base_model, ctc_model
base_cnn_model, ctc_cnn_model = create_cnn_model(input_size, output_size=py_list_size) 

his = ctc_cnn_model.fit(train_data[0], train_data[1], validation_split=0.1, batch_size=32, epochs=1000, callbacks=cb)  # callback的epoch都是对fit里的参数来说

#  保存模型结构及权重
ctc_cnn_model.save_weights(r'save_cnn_weights.h5')
with open(r'model_cnn_struct.json', 'w') as f:
    json_string = base_cnn_model.to_json()
    f.write(json_string)  # 保存模型信息
print('模型结构及权重已保存')

In [ ]:
# 加载权重
with open(r'model_cnn_struct.json') as f:
    model_cnn_struct = f.read()
test_cnn_model = keras.models.model_from_json(model_cnn_struct)
# model.summary()
test_cnn_model.load_weights(r'save_cnn_weights.h5')
# model = keras.models.load_model('all_model.h5')
print('模型已加载')
py_list = []
with open(r'pinyin_list.txt', 'r') as f:
    contents = f.readlines()
for line in contents:
    i = line.strip('\n')
    py_list.append(i)
print('py_list已加载')

# 对模型进行评价


def evaluate(kind, wavs, labels):
    data_num = len(wavs)
    error_cnt = 0
    for i in range(data_num):
        pre = test_cnn_model.predict(wavs[i])  # (1, 20, 11)
        pre_index, pre_label = decode_ctc(pre, py_list)  # ['5']
        try:
            pre_label = int(pre_label[0])
        except:
            pre_label = None
        label = int(py_list[labels[i]])
        if label != pre_label:
            error_cnt += 1
            print('真实标签：', label, '预测结果', pre_label)
    print('{}:样本数{}错误数{}准确率：{:%}'.format(
        kind, data_num, error_cnt, (1-error_cnt/data_num)))

evaluate('trian', train_wavs, train_labels)
evaluate('test', test_wavs, test_labels)

## LSTM

In [ ]:
# #LSTM minist分类
# from keras.datasets import mnist
# n_input = 28
# n_step = 28
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train = x_train.reshape(-1, n_step, n_input, 1)
# x_test = x_test.reshape(-1, n_step, n_input, 1)
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255

# y_train = keras.utils.to_categorical(y_train, n_classes)
# y_test = keras.utils.to_categorical(y_test, n_classes)
# inputs = Input(name='the_inputs', shape=(n_step, n_input, 1))
# inner = Reshape((n_step, n_input),name='Reshape_1')(inputs)
# # expected input data shape: (batch_size, timesteps, data_dim)
# lstm_1 = LSTM(128, kernel_initializer='he_normal', name='lstm1')(inner)
# d1 = Dense(10)(lstm_1)
# outputs = Activation('softmax')(d1)
# model = keras.Model(inputs, outputs)

# model.summary()
# model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# model.fit(x_train, y_train,batch_size=128,epochs=20,verbose=1,validation_data=(x_test, y_test))
# scores = model.evaluate(x_test, y_test, verbose=0)
# print('LSTM test score:', scores[0])
# print('LSTM test accuracy:', scores[1])

In [ ]:
# LSTM太差
def create_lstm_model(input_size, output_size):
    inputs = Input(name='the_inputs', shape=input_size)
    inner = Reshape((72, 64), name='Reshape_1')(inputs)
    # expected input data shape: (batch_size, timesteps, data_dim)
    lstm_1 = Bidirectional(LSTM(32, return_sequences=True,
                                kernel_initializer='he_normal', name='lstm1'))(inner)

    lstm_2 = Bidirectional(LSTM(128, return_sequences=True,
                                kernel_initializer='he_normal', name='lstm3'))(lstm_1)

    h5_2 = Dense(512, activation='relu', use_bias=True,
                 kernel_initializer='he_normal', name='Dense_4')(lstm_2)
    h5_3 = Dense(output_size, activation="relu", use_bias=True,
                 kernel_initializer='he_normal', name='Dense_5')(h5_2)  # (layer_h15)

    outputs = Activation('softmax', name='Activation_1')(h5_3)

    base_model = keras.Model(inputs=inputs, outputs=outputs)

    labels = Input(name='the_labels', shape=[None], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    # keras.Lambda(function, output_shape=None, mask=None, arguments=None)
    # 将任意表达式封装为 Layer 对象
    loss_out = Lambda(ctc_lambda, output_shape=(1,), name='ctc')(
        [labels, outputs, input_length, label_length])
    ctc_model = keras.Model(
        inputs=[labels, inputs, input_length, label_length], outputs=loss_out)

    opt = keras.optimizers.Adam(
        lr=0.0008, beta_1=0.9, beta_2=0.999, decay=0.01, epsilon=10e-8)
    # ctc_model=multi_gpu_model(ctc_model,gpus=2)
    ctc_model.compile(loss={'ctc': lambda y_true, output: output},
                      optimizer=opt, metrics=['accuracy'])

    return base_model, ctc_model


base_lstm_model, ctc_lstm_model = create_lstm_model(
    input_size=(72, 64, 1), output_size=py_list_size)
# ctc_lstm_model.summary()
train_data = get_train_data(
    train_wav_file_list, train_label_list, py_list, image_size)
his = ctc_lstm_model.fit(train_data[0], train_data[1], validation_split=0.1,
                         batch_size=32, verbose=0, epochs=1000, callbacks=cb)  # callback的epoch都是对fit里的参数来说

#  保存模型结构及权重
ctc_lstm_model.save_weights(r'save_lstm_weights.h5')
with open(r'model_lstm_struct.json', 'w') as f:
    json_string = base_lstm_model.to_json()
    f.write(json_string)  # 保存模型信息
print('模型结构及权重已保存')

# 加载权重
with open(r'model_lstm_struct.json') as f:
    model_lstm_struct = f.read()
test_lstm_model = keras.models.model_from_json(model_lstm_struct)

test_lstm_model.load_weights(r'save_lstm_weights.h5')
print('模型已加载')
py_list = []
with open(r'pinyin_list.txt', 'r') as f:
    contents = f.readlines()
for line in contents:
    i = line.strip('\n')
    py_list.append(i)
print('py_list已加载')

In [ ]:
# 对模型进行评价
def evaluate(kind, wavs, labels):
    data_num = len(wavs)
    error_cnt = 0
    for i in range(data_num):
        pre = test_lstm_model.predict(wavs[i])  # (1, 20, 11)
        pre_index, pre_label = decode_ctc(pre, py_list)  # ['5']
        try:
            pre_label = int(pre_label[0])
        except:
            pre_label = None
        label = int(py_list[labels[i]])
        if label != pre_label:
            error_cnt += 1
            print('真实标签：', label, '预测结果', pre_label)
    print('{}:样本数{}错误数{}准确率：{:%}'.format(
        kind, data_num, error_cnt, (1-error_cnt/data_num)))


# 训练集
train_wavs, train_labels = get_test_data(
    image_size, train_wav_file_list, train_label_list, py_list)
# 测试集
test_wavs, test_labels = get_test_data(
    image_size, test_wav_file_list, test_label_list, py_list)

evaluate('trian', train_wavs, train_labels)
evaluate('test', test_wavs, test_labels)

## 交叉熵损失函数的CRNN

In [40]:
def create_cross_model(input_size, output_size):
    inputs = Input(name='the_inputs', shape=input_size)
    # 1
    h1_1 = Conv2D(64, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_1-1')(inputs)
    h1_2 = BatchNormalization(name='BatchNormal_1-1')(h1_1)
    h1_3 = Conv2D(64, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_1-2')(h1_2)
    h1_4 = BatchNormalization(name='BatchNormal_1-2')(h1_3)
    h1_5 = MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", name='MaxPooling2D_1')(h1_4)
    # 2
    h2_1 = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_2-1')(h1_5)
    h2_2 = BatchNormalization(name='BatchNormal_2-1')(h2_1)
    h2_3 = Conv2D(128, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_2-2')(h2_2)
    h2_4 = BatchNormalization(name='BatchNormal_2-2')(h2_3)
    h2_5 = MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", name='MaxPooling2D_2')(h2_4)
    # 3
    h3_1 = Conv2D(256, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_3-1')(h2_5)
    h3_2 = BatchNormalization(name='BatchNormal_3-1')(h3_1)
    h3_3 = Conv2D(256, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_3-2')(h3_2)
    h3_4 = BatchNormalization(name='BatchNormal_3-2')(h3_3)
    h3_5 = MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", name='MaxPooling2D_3')(h3_4)
    # 4
    h4_1 = Conv2D(512, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_4-1')(h3_5)
    h4_2 = BatchNormalization(name='BatchNormal_4-1')(h4_1)
    h4_3 = Conv2D(512, (3, 3), use_bias=True, activation='relu', padding='same', kernel_initializer='he_normal', name='Conv2D_4-2')(h4_2)
    h4_4 = BatchNormalization(name='BatchNormal_4-2')(h4_3)
    # 由于声学模型网络结构原因（3个maxpooling层），我们的音频数据的每个维度需要能够被8整除。这里输入序列经过卷积网络后，长度缩短了8倍，因此我们训练实际输入的数据为wav_len//8。
    h5_1 = Reshape((9, int(input_size[1]//8*512)), name='Reshape_1')(h4_4)

    lstm_1 = LSTM(128, return_sequences=True, kernel_initializer='he_normal', name='lstm1')(h5_1)
    lstm_2 = LSTM(256, return_sequences=True, kernel_initializer='he_normal', name='lstm2')(lstm_1)
    h5_11 = Flatten()(lstm_2)
    h5_2 = Dense(512, activation='relu', use_bias=True, kernel_initializer='he_normal', name='Dense_1')(h5_11)
    h5_3 = Dense(output_size, activation="relu", use_bias=True, kernel_initializer='he_normal', name='Dense_2')(h5_2)#(layer_h15)
    outputs = Activation('softmax', name='Activation_1')(h5_3)
    base_model = keras.Model(inputs=inputs, outputs=outputs)

    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.09, beta_2=0.999, decay=0.1, epsilon=10e-8)
    # ctc_model=multi_gpu_model(ctc_model,gpus=2)
    base_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return base_model

crnn_cross_model = create_cross_model(input_size=(72, 64, 1), output_size=6) 

train_data = get_train_data(train_wav_file_list, train_label_list, py_list, image_size) 
from keras.utils import np_utils

l = np_utils.to_categorical(train_data[0]['the_labels'], num_classes=6)
his = crnn_cross_model.fit(train_data[0]['the_inputs'], l, validation_split=0.2, batch_size=32, verbose=0, epochs=1000, callbacks=cb)  # callback的epoch都是对fit里的参数来说

#  保存模型结构及权重
crnn_cross_model.save_weights(r'save_crnn_cross_weights.h5')
with open(r'model_crnn_cross_struct.json', 'w') as f:
    json_string = crnn_cross_model.to_json()
    f.write(json_string)  # 保存模型信息
print('模型结构及权重已保存')

Epoch 00030: early stopping
模型结构及权重已保存


In [41]:
# 加载权重
with open(r'model_crnn_cross_struct.json') as f:
  crnn_cross_struct = f.read()
test_crnn_cross_model = keras.models.model_from_json(crnn_cross_struct)
test_crnn_cross_model.load_weights(r'save_crnn_cross_weights.h5')
print('模型已加载')
py_list = []
with open(r'pinyin_list.txt', 'r') as f:
    contents = f.readlines()
for line in contents:
    i = line.strip('\n')
    py_list.append(i)
print('py_list已加载')
loss, accuracy = crnn_cross_model.evaluate(np.array(test_wavs).reshape((60, 72, 64, 1)), np_utils.to_categorical(np.array(test_labels),num_classes=6))

print('准确率：{:%}'.format(accuracy))

模型已加载
py_list已加载


ValueError: cannot reshape array of size 184320 into shape (60,72,64,1)

In [ ]:
# shown_offset = re.findall(re.compile(r'<ul.*?id="feedlist_id" shown-offset="(.*?)">', re.S), response.text)[0]


## 选择方差第二大的子载波

In [ ]:
def process_data(raw_file_path, save_path):
    X = {}
    for f in tqdm(os.listdir(raw_file_path)):
        if f.endswith('.dat'):
            file_name = os.path.join(raw_file_path, f)
            extracted_data = extract_csi(file_name)
            print('processing {} the length of this file is:{}'.format(
                file_name, len(extracted_data)))
            tx, rx, sub = extracted_data[0]['csi'].shape
            data_csi = np.zeros(
                (len(extracted_data), tx, rx, sub), dtype=np.complex64)
            for i in range(len(extracted_data)):
                data_csi[i] = get_scaled_csi(extracted_data[i])
            data_csi = np.clip(np.abs(np.squeeze(data_csi)), 1e-8,
                               1e100)[:, :, :2, :].reshape(-1, 120)   # (205, 2, 2, 30)
            data = np.zeros((data_csi.shape[0], 120))
            var = []
            for i in range(120):
                data_csi_sub = data_csi[:, i]
                b, a = signal.butter(5, 4*2/30, 'low')
                carrier_data = signal.lfilter(b, a, data_csi_sub)  # N*1
                data[:, i] = carrier_data

                # length = len(carrier_data)
                # var_temp = np.var(carrier_data[length//5:3*length//5])
                # var.append(var_temp)

            #data = data_csi[:, np.argsort(var)[1]]
            # print(np.argsort(var)[1])
            scio.savemat(os.path.join(
                save_path, f.split('.')[0]+'.mat'), {'csi': data})
            X[f] = data
    print('all raw file processed')
    return X


train_raw_file_path = r'/content/drive/My Drive/SR_CSI/Gestures_data/train'
train_save_path = r'/content/drive/My Drive/SR_CSI/Gestures_data/train'
X_train = process_data(train_raw_file_path, train_save_path)
test_raw_file_path = r'/content/drive/My Drive/SR_CSI/Gestures_data/test'
test_save_path = r'/content/drive/My Drive/SR_CSI//Gestures_data/test'
X_test = process_data(test_raw_file_path, test_save_path)

In [ ]:
# 重新构造mat文件选择最佳子载波上的CSI
best_index_list = [58, 1, 29, 29, 28, 29]

#
for f in tqdm(os.listdir(train_save_path)):
    if f.endswith('.mat'):
      file_name = os.path.join(train_save_path, f)
      activate_index = int(re.findall(re.compile(r'csi-s1-e1-a(.*?)-.*?.mat', re.S), f)[0])
      csi = np.squeeze(scio.loadmat(file_name)['csi'])
      best_csi = csi[:, best_index_list[activate_index-1]]     
      scio.savemat(file_name, {'csi': best_csi})
for f in tqdm(os.listdir(test_save_path)):
    if f.endswith('.mat'):
      file_name = os.path.join(test_save_path, f)
      activate_index = int(re.findall(re.compile(r'csi-s1-e1-a(.*?)-.*?.mat', re.S), f)[0])
      best_csi = np.squeeze(scio.loadmat(file_name)['csi'])[:, best_index_list[activate_index-1]]
      scio.savemat(file_name, {'csi': best_csi})

In [ ]:
train_file_path = train_save_path
train_label_file_list, train_wav_file_list = get_file_list(train_file_path)

train_label_list = get_label_data(train_label_file_list)

# 用迭代器的时候需要
# train_wav_file_list, validate_wav_file_list, train_label_list, validate_label_list = train_test_split(train_wav_file_list, train_label_list, test_size=0.2, random_state=0)

# 每个文件拼音标签的集合
with open('label_list.txt', 'w') as f:  
    f.write('\n'.join(train_label_list))  
py_list = gen_py_list(train_label_list)  # 拼音的集合

with open('pinyin_list.txt', 'w') as f:
    f.write('\n'.join(py_list))  # 保存拼音列表

train_file_nums = len(train_wav_file_list)
# validate_file_nums = len(validate_wav_file_list)

py_list_size = len(py_list)  # 模型输出的维度
print('py_list:', py_list)
print('train files amount:', len(train_label_list), train_file_nums, 'label amount:', py_list_size)#, 'validate files amount:', len(validate_label_list), validate_file_nums,)

# 测试数据
test_file_path = test_save_path # r'/content/drive/My Drive/data_thchs30'
test_label_file_list, test_wav_file_list = get_file_list(test_file_path)

test_label_list = get_label_data(test_label_file_list)
print('test files amount:',len(test_label_file_list), len(test_wav_file_list))
test_data_num = len(test_label_file_list)

In [ ]:
base_sec_model, ctc_sec_model = create_model(input_size, output_size=py_list_size) 

train_data = get_train_data(train_wav_file_list, train_label_list, py_list, image_size)


cb = []
cb.append(keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0))
# 当监测值不再改善时，该回调函数将中止训练可防止过拟合
cb.append(keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, verbose=1, mode='auto'))
# his = ctc_model.fit_generator(train_batch_gen, verbose=1, steps_per_epoch=train_file_nums//batch_size, validation_data=validate_batch_gen, validation_steps=validate_file_nums//batch_size, epochs=1000, callbacks=cb)  


his = ctc_sec_model.fit(train_data[0], train_data[1], validation_split=0.1, batch_size=32, verbose=0, epochs=1000, callbacks=cb)  # callback的epoch都是对fit里的参数来说

#  保存模型结构及权重
ctc_sec_model.save_weights(r'save_sec_weights.h5')
with open(r'model_sec_struct.json', 'w') as f:
    json_string = base_sec_model.to_json()
    f.write(json_string)  # 保存模型信息
print('模型结构及权重已保存')

In [ ]:
# 加载权重
with open(r'model_sec_struct.json') as f:
  model_struct = f.read()
test_sec_model = keras.models.model_from_json(model_struct)
test_sec_model.load_weights(r'save_sec_weights.h5')
# model = keras.models.load_model('all_model.h5')
print('模型已加载')
py_list = []
with open(r'pinyin_list.txt', 'r') as f:
    contents = f.readlines()
for line in contents:
    i = line.strip('\n')
    py_list.append(i)
print('py_list已加载')

# 对模型进行评价
def evaluate(kind, wavs, labels):
  data_num = len(wavs)
  error_cnt = 0
  for i in range(data_num):
    pre = test_sec_model.predict(wavs[i]) # (1, 20, 11)
    pre_index, pre_label = decode_ctc(pre, py_list) # ['5']
    try:
      pre_label = int(pre_label[0])
    except:
      pre_label = None
    label = int(py_list[labels[i]])
    if label != pre_label:
      error_cnt += 1
      print('真实标签：', label, '预测结果', pre_label)
  print('{}:样本数{}错误数{}准确率：{:%}'.format(kind, data_num, error_cnt, (1-error_cnt/data_num)))

# 训练集
train_wavs, train_labels = get_test_data(image_size, train_wav_file_list, train_label_list, py_list)
# 测试集
test_wavs, test_labels = get_test_data(image_size, test_wav_file_list, test_label_list, py_list)

evaluate('trian', train_wavs, train_labels)
evaluate('test', test_wavs, test_labels)

## 所有子载波求平均

In [ ]:
def process_data(raw_file_path, save_path):
  X={}
  for f in tqdm(os.listdir(raw_file_path)):
    if f.endswith('.dat'):
      file_name = os.path.join(raw_file_path, f)
      extracted_data = extract_csi(file_name)
      print('processing {} the length of this file is:{}'.format(file_name, len(extracted_data)))
      tx, rx, sub = extracted_data[0]['csi'].shape
      data_csi = np.zeros((len(extracted_data), tx, rx, sub), dtype=np.complex64)
      for i in range(len(extracted_data)):
        data_csi[i] = get_scaled_csi(extracted_data[i])
      data_csi = np.clip(np.abs(np.squeeze(data_csi)), 1e-8, 1e100)[:,:,:2,:].reshape(-1, 4, 30)   # (205, 2, 2, 30)
      data = []
      for i in range(4):
        data_ave = np.average(data_csi[:, i, :],axis=1)
        data.extend(data_ave)
      scio.savemat(os.path.join(save_path, f.split('.')[0]+'.mat'), {'csi': data})
      X[f]=data
  print('all raw file processed')
  return X

train_raw_file_path = r'/content/drive/My Drive/SR_CSI/Gestures_data/train'
train_save_path = r'/content/drive/My Drive/SR_CSI/Gestures_data/train'
X_train = process_data(train_raw_file_path, train_save_path)
test_raw_file_path = r'/content/drive/My Drive/SR_CSI/Gestures_data/test'
test_save_path = r'/content/drive/My Drive/SR_CSI//Gestures_data/test'
X_test = process_data(test_raw_file_path, test_save_path)

In [ ]:
train_file_path = train_save_path
train_label_file_list, train_wav_file_list = get_file_list(train_file_path)

train_label_list = get_label_data(train_label_file_list)

# 用迭代器的时候需要
# train_wav_file_list, validate_wav_file_list, train_label_list, validate_label_list = train_test_split(train_wav_file_list, train_label_list, test_size=0.2, random_state=0)

# 每个文件拼音标签的集合
with open('label_list.txt', 'w') as f:  
    f.write('\n'.join(train_label_list))  
py_list = gen_py_list(train_label_list)  # 拼音的集合

with open('pinyin_list.txt', 'w') as f:
    f.write('\n'.join(py_list))  # 保存拼音列表

train_file_nums = len(train_wav_file_list)
# validate_file_nums = len(validate_wav_file_list)

py_list_size = len(py_list)  # 模型输出的维度
print('py_list:', py_list)
print('train files amount:', len(train_label_list), train_file_nums, 'label amount:', py_list_size)#, 'validate files amount:', len(validate_label_list), validate_file_nums,)

# 测试数据
test_file_path = test_save_path # r'/content/drive/My Drive/data_thchs30'
test_label_file_list, test_wav_file_list = get_file_list(test_file_path)

test_label_list = get_label_data(test_label_file_list)
print('test files amount:',len(test_label_file_list), len(test_wav_file_list))
test_data_num = len(test_label_file_list)

In [ ]:
base_ave_model, ctc_ave_model = create_model(input_size=(None, 256, 1), output_size=py_list_size) 

train_data = get_train_data(train_wav_file_list, train_label_list, py_list, image_size=256)

his = ctc_ave_model.fit(train_data[0], train_data[1], validation_split=0.1, batch_size=32, verbose=1, epochs=1000, callbacks=cb)  # callback的epoch都是对fit里的参数来说

#  保存模型结构及权重
ctc_ave_model.save_weights(r'save_ave_weights.h5')
with open(r'model_ave_struct.json', 'w') as f:
    json_string = base_ave_model.to_json()
    f.write(json_string)  # 保存模型信息
print('模型结构及权重已保存')

In [ ]:
# 加载权重
with open(r'model_ave_struct.json') as f:
  model_struct = f.read()
test_ave_model = keras.models.model_from_json(model_struct)
test_ave_model.load_weights(r'save_ave_weights.h5')
# model = keras.models.load_model('all_model.h5')
print('模型已加载')
py_list = []
with open(r'pinyin_list.txt', 'r') as f:
    contents = f.readlines()
for line in contents:
    i = line.strip('\n')
    py_list.append(i)
print('py_list已加载')

# 对模型进行评价
def evaluate(kind, wavs, labels):
  data_num = len(wavs)
  error_cnt = 0
  for i in range(data_num):
    pre = test_sec_model.predict(wavs[i]) # (1, 20, 11)
    pre_index, pre_label = decode_ctc(pre, py_list) # ['5']
    try:
      pre_label = int(pre_label[0])
    except:
      pre_label = None
    label = int(py_list[labels[i]])
    if label != pre_label:
      error_cnt += 1
      print('真实标签：', label, '预测结果', pre_label)
  print('{}:样本数{}错误数{}准确率：{:%}'.format(kind, data_num, error_cnt, (1-error_cnt/data_num)))

# 训练集
train_wavs, train_labels = get_test_data(image_size, train_wav_file_list, train_label_list, py_list)
# 测试集
test_wavs, test_labels = get_test_data(image_size, test_wav_file_list, test_label_list, py_list)

evaluate('trian', train_wavs, train_labels)
evaluate('test', test_wavs, test_labels)

#### ROC、AUC曲线 

In [ ]:
# from sklearn.datasets import make_classification
# from sklearn.preprocessing import label_binarize
# from keras.models import Sequential
# from keras.layers import Dense
# import numpy as np
# from scipy import interp
# import matplotlib.pyplot as plt
# from itertools import cycle
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_curve, auc

# # 标签共三类
# n_classes = py_list_size-1

# X, y = make_classification(n_samples=80000, n_features=20, n_informative=3, n_redundant=0, n_classes=n_classes,
#     n_clusters_per_class=2)
# # print(X.shape, y.shape)
# # print(X[0], y[0])
# # (80000, 20) (80000,)
# # [-1.90920853 -1.30052757 -0.76903467 -3.2546519  -0.02947816  0.14105006
# #   0.43556031 -0.81300607 -0.94553296 -0.92774495  1.49041451 -0.4443121
# #  -1.16342165 -0.32997815 -1.02907045 -0.39950447 -0.711287    0.51382424
# #   2.88822258 -2.0935274 ] 
# # 1

# # Binarize the output相当于one_hot
# y = label_binarize(y, classes=[0, 1, 2])
# # print(y.shape, y[0])
# # (80000, 3) [0 1 0]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
# model = Sequential()
# model.add(Dense(20, input_dim=20, activation='relu'))
# model.add(Dense(40, activation='relu'))
# model.add(Dense(3, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=1, batch_size=100, verbose=1)

# y_pred = model.predict(X_test)
# # print(y_pred.shape)
# # (40000, 3)

# # Compute ROC curve and ROC area for each class
# fpr = dict()
# tpr = dict()
# roc_auc = dict()
# for i in range(n_classes):
#     # scores = np.array([0.1, 0.4, 0.35, 0.8])
#     # fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=2)
#     # y 就是标准值，scores 是每个预测值对应的阳性概率，比如0.1就是指第一个数预测为阳性的概率为0.1，很显然，
#     # y 和 socres应该有相同多的元素，都等于样本数。pos_label=2 是指在y中标签为2的是标准阳性标签，其余值是阴性。
#     # 接下来选取一个阈值计算TPR/FPR,阈值的选取规则是在scores值中从大到小的以此选取，于是第一个选取的阈值是0.8
#     # label=[1,1,2,2] scores=[0.1,0.4,0.35,0.8] thresholds=[0.8,0.4,0.35,0.1] 以threshold为0.8为例，将0.8与
#     # scores 中所有值比较大小得到预测值，[0,0,0,1].对于label中两个1，其概率分别为0.1，0.4，小于阈值0.8，判定为
#     # 负样本，而他们的label是1，说明他们确实是负样本，判断正确，是两个TN；两个2，对应概率为0.35，0.8，0.35小于
#     # 0.8，判定为负样本，但是label是2，应该是个正样本，所以这是个FN；最后0.8>=0.8,这是个TP，所以最后的结果是
#     # ：1个TP，2个TN，1个FN，0个FP
#     fpr[i], tpr[i], thresholds = roc_curve(y_test[:, i], y_pred[:, i])  # (40000,)
#     # print(fpr[i].shape)# (5491,)# (6562,)# (4271,)
#     roc_auc[i] = auc(fpr[i], tpr[i])
    

# # 计算microROC曲线和ROC面积 
# # .ravel()将多维数组转换为一维数组
# fpr["micro"], tpr["micro"]  , thresholds = roc_curve(y_test.ravel(), y_pred.ravel())  #  (120000,)
# roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# # 计算macroROC曲线和ROC面积
# # 首先，汇总所有的假阳性率
# # np.unique() 该函数是去除数组中的重复数字，并进行排序之后输出。
# # print(np.concatenate([fpr[i] for i in range(n_classes)]).shape) (16324,)
# all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))  # (7901,)
# # 然后插值所有的ROC曲线在这一点
# # np.zeros_like() 这个函数的意思就是生成一个和你所给数组a相同shape的全0数组。
# mean_tpr = np.zeros_like(all_fpr)
# for i in range(n_classes):
#     mean_tpr += interp(all_fpr, fpr[i], tpr[i])
    
# # 最后求平均值并计算AUC
# mean_tpr /= n_classes
# fpr["macro"] = all_fpr
# tpr["macro"] = mean_tpr
# roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# # Plot all ROC curves
# plt.figure(1)
# plt.plot(fpr["micro"], tpr["micro"], color='deeppink', linestyle=':', linewidth=4,
#          label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]))

# plt.plot(fpr["macro"], tpr["macro"],color='navy', linestyle=':', linewidth=4,
#          label='macro-average ROC curve (area = {0:0.2f})'.format(roc_auc["macro"]))

# colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
# for i, color in zip(range(n_classes), colors):
#     plt.plot(fpr[i], tpr[i], color=color, linewidth=2,
#              label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

# plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Some extension of Receiver Operating Characteristic to multi-class')
# plt.legend(loc='best')
# plt.show()


# # Zoom in view of the upper left corner.
# plt.figure(2)
# plt.xlim(0, 0.2)
# plt.ylim(0.8, 1)
# plt.plot(fpr["micro"], tpr["micro"],color='deeppink', linestyle=':', linewidth=4,
#          label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]))

# plt.plot(fpr["macro"], tpr["macro"],color='navy', linestyle=':', linewidth=4,
#          label='macro-average ROC curve (area = {0:0.2f})'.format(roc_auc["macro"]))

# colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
# for i, color in zip(range(n_classes), colors):
#     plt.plot(fpr[i], tpr[i], color=color, linewidth=2,
#              label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

# plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC curve (zoomed in at top left)')
# plt.legend(loc='best')
# plt.show()

#### 混淆矩阵 

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix
# def plot_confusion_matrix(title, y_true, y_pred, labels):
#     cm = confusion_matrix(y_true, y_pred)
    
#     # np.newaxis的作用就是在这一位置增加一个一维，这一位置指的是np.newaxis所在的位置，比较抽象，需要配合例子理解。
#     # x1 = np.array([1, 2, 3, 4, 5])
#     # the shape of x1 is (5,)
#     # x1_new = x1[:, np.newaxis]
# # now, the shape of x1_new is (5, 1)


#     cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#     # print (cm, '\n\n', cm_normalized)
#     # [[1 0 0 0 0]                           
#     #  [0 1 0 0 0]
#     #  [0 0 1 0 0]
#     #  [0 0 0 1 0]
#     #  [0 0 0 0 1]]

#     #  [[1. 0. 0. 0. 0.]
#     #  [0. 1. 0. 0. 0.]
#     #  [0. 0. 1. 0. 0.]
#     #  [0. 0. 0. 1. 0.]
#     #  [0. 0. 0. 0. 1.]]
#     tick_marks = np.array(range(len(labels))) + 0.5
#     #  [0.5 1.5 2.5 3.5 4.5 5.5]
#     np.set_printoptions(precision=2)
    
#     plt.figure(figsize=(10, 8), dpi=120)
#     ind_array = np.arange(len(labels))
#     x, y = np.meshgrid(ind_array, ind_array)
#     # print(ind_ａrray, '\n\n', x, '\n\n', y)
#     # [0 1 2 3 4 5] 

#     #  [[0 1 2 3 4 5]
#     #  [0 1 2 3 4 5]
#     #  [0 1 2 3 4 5]
#     #  [0 1 2 3 4 5]
#     #  [0 1 2 3 4 5]
#     #  [0 1 2 3 4 5]] 

#     #  [[0 0 0 0 0 0]
#     #  [1 1 1 1 1 1]
#     #  [2 2 2 2 2 2]
#     #  [3 3 3 3 3 3]
#     #  [4 4 4 4 4 4]
#     #  [5 5 5 5 5 5]]
#     intFlag = 0 # 标记在图片中对文字是整数型还是浮点型
#     for x_val, y_val in zip(x.flatten(), y.flatten()):
#         # plt.text()函数用于设置文字说明。

#         if (intFlag):
#             c = cm[y_val][x_val]
#             plt.text(x_val, y_val, "%d" % (c,), color='red', fontsize=8, va='center', ha='center')

#         else:
#             c = cm_normalized[y_val][x_val]
#             if (c > 0.01):
#                 plt.text(x_val, y_val, "%0.2f" % (c,), color='red', fontsize=7, va='center', ha='center')
#             else:
#                 plt.text(x_val, y_val, "%d" % (0,), color='red', fontsize=7, va='center', ha='center')
#     cmap = plt.cm.binary
#     if(intFlag):
#         plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     else:
#         plt.imshow(cm_normalized, interpolation='nearest', cmap=cmap)
#     plt.gca().set_xticks(tick_marks, minor=True)
#     plt.gca().set_yticks(tick_marks, minor=True)
#     plt.gca().xaxis.set_ticks_position('none')
#     plt.gca().yaxis.set_ticks_position('none')
#     plt.grid(True, which='minor', linestyle='-')
#     plt.gcf().subplots_adjust(bottom=0.15)
#     plt.title(title)
#     plt.colorbar()
#     xlocations = np.array(range(len(labels)))
#     plt.xticks(xlocations, labels, rotation=90)
#     plt.yticks(xlocations, labels)
#     plt.ylabel('Index of True Classes')
#     plt.xlabel('Index of Predict Classes')
#     plt.savefig('confusion_matrix.jpg', dpi=300)
#     plt.show()
# title='Confusion Matrix'
# labels = ['A', 'B', 'C', 'F', 'G']
# y_true = 
# y_pred = [1, 2, 3, 4, 5]# np.loadtxt(r'/home/dingtom/b.txt')
# plot＿confusion_matrix(title, y_true,y_pred, labels)